# Spark demo : Part I #

In this lab, we will introduce some of Spark's capabilities.

In [0]:
sc

In [0]:
my_rdd = sc.textFile('./data/ai.stackexchange.com/Posts.xml')
my_rdd

In [0]:
??sc.textFile

In [0]:
my_rdd \
    .take(5)
    #     .sample(True, .01) \
#     .take(5)

In [0]:
my_rdd.count()

In [0]:
import re

def is_post_row(line):
    return re.match('  <row Id=', line) != None

# my_rdd.map(is_post_row).take(20)

# my_rdd.filter(lambda x: is_post_row(x)==False).collect()

row_rdd = my_rdd \
    .filter(is_post_row)

test_rows = row_rdd.take(20)

row_rdd.count()

In [0]:
is_post_row("hello!")

In [0]:
is_post_row(test_rows[10])

In [0]:
test_rows

In [0]:
def extract_post_type_id(line):
    matches = re.findall('PostTypeId="(\d+)"', line)
    return int(matches[0])

my_rdd \
    .filter(is_post_row) \
    .map(extract_post_type_id) \
    .take(10)

In [0]:
my_rdd \
    .filter(is_post_row) \
    .map(extract_post_type_id)

In [0]:
type_counts = my_rdd \
    .filter(is_post_row) \
    .map(extract_post_type_id) \
    .map(lambda x: (x, 1)) \
    .reduceByKey(lambda x, y: x+y) \
    .sortByKey() \
    .collect()
    
for tc in type_counts:
    print tc

In [0]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

x, y = zip(*type_counts)
plt.bar(x, y)
plt.title("PostTypeID frequency counts")
plt.xlabel("PostTypeID")
plt.ylabel("Frequency")

In [0]:
import datetime

def extract_date(line):
    matches = re.findall('CreationDate=\"(\d{4})-(\d{2})-(\d{2})', line)
    y, m, d = matches[0]
    dt = datetime.date(int(y), int(m), int(d))
    return dt

[extract_date(row) for row in test_rows]

date_counts = row_rdd \
    .map(extract_date) \
    .map(lambda date: (date, 1)) \
    .reduceByKey( lambda x, y: x+y) \
    .sortByKey() \
    .collect()

print date_counts[:10]

In [0]:
x, y = zip(*date_counts)
plt.bar(x, y)
plt.title("Posts by day")
plt.xlabel("Date")
plt.ylabel("Frequency")

### Exercise 1 ###
- Write a regular-expression-based function to parse the "Score" field in each row.
- What is the range of values for Score?
- Which are most common?
- Create a bar chart to show the distribution of Score frequencies.

### Exercise 2 ###
- Write a regular-expression-based function to parse the "ViewCount" field in each row.
- Write an RDD to compute the average ViewCount.
- Create a line plot to show the distribution of ViewCount frequencies.

### Exercise 3 ###
- Write a new RDD to parse ViewCount, Score, FavoriteCount, and Date, all within the row value.
- Create a scatter plot to show the relationship of ViewCounts and Scores.
- Write a different RDD to compute the average ViewCount, grouped by Score.